In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
2+2

4

In [80]:
import numpy as np


In [83]:
from pylab import *
import matplotlib.pyplot as plt
import os
import pyAgrum as gum
import pyAgrum.lib.notebook as gnb

print('imported')

bn=gum.BayesNet('backdoor')
Z,X,Y=[ bn.add(name, 2) for name in "ZXY" ] 
print (bn)

all_nodes=['X','Z', 'Y'] 

bn.addArc(Z,Y)
bn.addArc(Z,X)
bn.addArc(X,Y)
gnb.flow.row(bn)

bn

imported
BN{nodes: 3, arcs: 0, domainSize: 8, dim: 3, mem: 48o}


(pyAgrum.BayesNet<double>@0x561dd0ae1760) BN{nodes: 3, arcs: 3, domainSize: 8, dim: 7, mem: 112o}

In [ ]:
for nd in all_nodes:
    bn.generateCPT(nd)
    print(bn.cpt(nd))
    # print(nd)


In [87]:
ie=gum.LazyPropagation(bn)
ie.makeInference()

# ie.evidenceJointImpact(['Z','X'], [])
ie.evidenceJointImpact(['Z','X', 'Y'], [])

(pyAgrum.Potential<double>@0x561dce6a7d90) 
             ||  X                |
Z     |Y     ||0        |1        |
------|------||---------|---------|
0     |0     || 0.1019  | 0.0001  |
1     |0     || 0.0543  | 0.0059  |
0     |1     || 0.2773  | 0.0086  |
1     |1     || 0.1394  | 0.4125  |

In [88]:
ie=gum.LazyPropagation(bn)
ie.makeInference()

ie.evidenceJointImpact(['X'], [])

(pyAgrum.Potential<double>@0x561dd0c14eb0) 
  X                |
0        |1        |
---------|---------|
 0.5728  | 0.4272  |

# Causal Effect

In [91]:
out= bn.cpt('Z')* bn.cpt('Y')
# out
true_doX_cpt = out.margSumOut('Z')
true_doX_cpt

(pyAgrum.Potential<double>@0x561dce9c54a0) 
      ||  Y                |
X     ||0        |1        |
------||---------|---------|
0     || 0.2757  | 0.7243  |
1     || 0.0134  | 0.9866  |

In [92]:
true_doX={}
for x in true_doX_cpt.loopIn():
    # print(x.todict(), true_doX_cpt[x])
    true_doX[tuple(x.todict().values())]= true_doX_cpt[x]

true_doX

{(0, 0): 0.27568764613056196,
 (1, 0): 0.7243123538694383,
 (0, 1): 0.013366143088356817,
 (1, 1): 0.9866338569116433}

# Data generation

In [93]:
g=gum.BNDatabaseGenerator(bn)
N= 10000
g.drawSamples(N,{})
dataset= g.to_pandas()

In [94]:

dataset = np.array(dataset).astype(int)
type(dataset)
unique_rows, counts = np.unique(dataset, axis=0, return_counts=True)
joint_distribution = counts / len(dataset)

# Print the unique rows and their corresponding probabilities
for row, prob in zip(unique_rows, joint_distribution):
    print(f"Row: {row}, Probability: {prob:.4f}")

Row: [0 0 0], Probability: 0.1059
Row: [0 0 1], Probability: 0.2714
Row: [0 1 0], Probability: 0.0004
Row: [0 1 1], Probability: 0.0082
Row: [1 0 0], Probability: 0.0523
Row: [1 0 1], Probability: 0.1381
Row: [1 1 0], Probability: 0.0067
Row: [1 1 1], Probability: 0.4170


In [95]:
import numpy as np



# N=10000
# sampleZ = np.random.binomial(n=1, p=pz[id,1], size=N)
# sampleX_Z = np.random.binomial(n=1, p=px_z[id,1], size=N)
# sampleY_XZ = np.random.binomial(n=1, p=py_xz[id,1], size=N)

 


sampleZ = torch.nn.functional.one_hot(torch.tensor(dataset[:,0]), num_classes=2).float()  
sampleX = torch.nn.functional.one_hot(torch.tensor(dataset[:,1]), num_classes=2).float()
sampleY = torch.nn.functional.one_hot(torch.tensor(dataset[:,2]), num_classes=2).float()




# Training code

In [96]:
import torch
import torch.nn as nn
import torch.optim as optim


def sample_gumbel(shape, eps=1e-20):
    U = torch.rand(shape)
    return -torch.log(-torch.log(U + eps) + eps)


def gumbel_softmax_sample(logits, temperature, gumbel_noise):

    if gumbel_noise==None:
        gumbel_noise= sample_gumbel(logits.size())

    y = logits + gumbel_noise
    return F.softmax(y / temperature, dim=-1)


def gumbel_softmax(logits, temperature, gumbel_noise=None, hard=False):
    """
    ST-gumple-softmax
    input: [*, n_class]
    return: flatten --> [*, n_class] an one-hot vector
    """
    output_dim =logits.shape[1]
    y = gumbel_softmax_sample(logits, temperature, gumbel_noise)

    if not hard:
        ret = y.view(-1, output_dim)
        return ret

    shape = y.size()
    _, ind = y.max(dim=-1)
    y_hard = torch.zeros_like(y).view(-1, shape[-1])
    y_hard.scatter_(1, ind.view(-1, 1), 1)
    y_hard = y_hard.view(*shape)
    # Set gradients w.r.t. y_hard gradients w.r.t. y
    y_hard = (y_hard - y).detach() + y
    # ret = y_hard.view(-1, output_dim)
    ret = y_hard.view(-1, output_dim)
    return ret


class SampleGenerator(nn.Module):
    def __init__(self, input_size, output_size=2):
        super(SampleGenerator, self).__init__()
        hidden_size=50
        # Define learnable latent parameter (no input needed)
        self.latent = nn.Parameter(torch.randn(N, hidden_size))
        
        # if input_size==0:
        input_size= input_size+ hidden_size
        
        # Define the layers
        self.fc0 = nn.Linear(input_size, hidden_size)  # Hidden layer
        self.fc1 = nn.Linear(hidden_size, hidden_size)  # Hidden layer
        self.fc2 = nn.Linear(hidden_size, output_size)  # Output layer to predict logits

    def forward(self, parents, temperature=1.0, hard=False):
        

        if parents==None:
            parents=self.latent
        else:
            parents= torch.tensor(parents)
            parents= torch.cat([parents, self.latent], dim=1)
        
        # Use the learnable latent vector
        x= torch.relu(self.fc0(parents))
        x = torch.relu(self.fc1(x))
        logits = torch.relu(self.fc2(x))
                
        
        #
        # logits = logits / torch.max(logits)  # normalize logits
        #
        
        # Apply Gumbel-Softmax to generate differentiable one-hot like vectors
        # gumbel_softmax_output = nn.functional.gumbel_softmax(logits, tau=temperature, hard=hard, eps=1e-10)
        
        gumbel_softmax_output= gumbel_softmax(logits , temperature, None, False)
        
        
        output=gumbel_softmax_output
        if torch.isnan(torch.max(output)):
            print('--->',output.shape, ' logists-->', torch.max(logits),torch.min(logits), ' tau-->', temperature)
        
        
        return gumbel_softmax_output


In [97]:

def marginal(input, freq):
    
    if input==None:
        comb=freq
    else:
        # inp= torch.argmax(input, dim=1).view(-1,1)
        comb= torch.cat([freq, input], dim=1)
            
    unique_rows, counts = np.unique(np.array(comb), axis=0, return_counts=True)
    joint_distribution = counts / len(dataset)
    
    rdict={}
    for row, prob in zip(unique_rows, joint_distribution):
        rdict[tuple(row)]= prob
        # print(f"Row: {row}, Probability: {prob:.4f}")
    
    return rdict




def train(input, targets, model, name, optimize='maximize', lambdaa=[0.1, 0.1]):

    
    # Loss function
    criterion = nn.CrossEntropyLoss()
    
    # Optimizer
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    # Training loop (for one sample per forward pass)
    for epoch in range(2000):
        optimizer.zero_grad()  # Zero the gradients
    
        # Forward pass to generate sample
        output = model(input, temperature=1.0, hard=False)  # Model generates sample
    
        # Compute loss
        loss = criterion(output, targets) #P(Y|X,Z)
        
        if torch.isnan(loss):
            print(torch.max(output), torch.min(output))
            break
            
        
        
        
        # Causal efffect loss
        if name=='Y' and optimize!='none':
            if optimize=='maximize':
                targAll= torch.nn.functional.one_hot(torch.ones(N).to(torch.int64), num_classes=2).float()
            elif optimize=='minimize':
                targAll= torch.nn.functional.one_hot(0* torch.ones(N).to(torch.int64), num_classes=2).float()
            
            # do(X=0) # P(Y=1|do(X=0)) to maximize;
            X= torch.nn.functional.one_hot(torch.zeros(N).to(torch.int64), num_classes=2).float()
            inputY= torch.cat([sampleZ, X], dim=1)  # X=0, Z~ P(Z) D[Z]
            outY0= model(inputY, temperature=1.0, hard=False)    # P(Y|x=0,Z)  of \sum_z P(z) P(Y|x=0,z)
            loss0 = criterion(outY0, targAll) 
            
            # 
            # 10000 X 2 : outY0: [0.1 , 0.9], [0.8, 0.2] ->  targAll: [0,1], [0,1]
            
            
            # do(X=1)     # P(Y=1|do(X=1)) to maximize;
            X= torch.nn.functional.one_hot(torch.ones(N).to(torch.int64), num_classes=2).float()
            inputY= torch.cat([sampleZ, X], dim=1)
            outY1= model(inputY, temperature=1.0, hard=False)
            loss1 = criterion(outY1, targAll)
            
                        
            # if torch.isnan(loss0):
            #     print(torch.max(outY0), torch.min(outY0))
            #     break
            #     
            #     
            # if torch.isnan(loss1):
            #     print(torch.max(outY1), torch.min(outY1))
            #     break
            
                                
            # loss+= 0.1*(loss0 + loss1)  # why optimizing both is wrong?  Check Bengio's paper.
            loss+= lambdaa[0]*(loss0) + lambdaa[1]*(loss1) 
            # loss+= lambdaa*(loss1)
            
        
        #
        
        # Backpropagation and optimizer step
        loss.backward()
        optimizer.step()
    
        if (epoch+1) % 50 == 0:
            
            freq= torch.argmax(output, dim=1).view(-1,1)
            rdict= marginal(input, freq)
            
            
            freq= torch.argmax(targets, dim=1).view(-1,1)
            tdict= marginal(input, freq)
                
            tvd=0
            for key in rdict:
                tvd+= abs(rdict[key]- tdict[key])
                
            
            ###
            if name=='Y':
                X= torch.nn.functional.one_hot(torch.zeros(N).to(torch.int64), num_classes=2).float()
                inputY= torch.cat([sampleZ, X], dim=1)
                outY= model(inputY, temperature=1.0, hard=False)  
                outY= torch.argmax(outY, dim=1).view(-1,1)
                doX0= marginal(None, outY)
                
                newDict={}
                for key in doX0:
                    newDict[ key+(0,)]  = doX0[key]
                doX0= newDict
                
                # tvdx0=0
                # for key in doX0:
                #     tvdx0+= abs(doX0[key]- true_doX0[key])
                
                
                ###
            
                X= torch.nn.functional.one_hot(torch.ones(N).to(torch.int64), num_classes=2).float()
                inputY= torch.cat([sampleZ, X], dim=1)
                outY= model(inputY, temperature=1.0, hard=False)  
                outY= torch.argmax(outY, dim=1).view(-1,1)
                doX1= marginal(None, outY)
                
                newDict={}
                for key in doX1:
                    newDict[ key+(1,)]  = doX1[key]
                doX1= newDict
                
                
                pred_doX= {**doX0, **doX1}
                
                tvdDox=0
                for key in doX1:
                    tvdDox+= abs(pred_doX[key]- true_doX[key])
                    
                
                # print(f'Causal effect: P(Y|do(X=0):{doX0} & P(Y|do(X=1):{doX1}')
                # print(f'-->{pred_doX} and {true_doX}')
            ###
            
            print(f'Epoch [{epoch+1}/1000], Loss: {loss.item():.4f} obsTVD:{tvd/2:0.4f} iTVD:{tvdDox/2:0.4f}')
            
    
    print("Training complete!")
    
    print('learned', rdict)
    print('True', tdict)



    return model

# Train Y

In [104]:
input= torch.cat([sampleZ, sampleX], dim=1)
targets = sampleY
input_size=4

modelY = SampleGenerator(input_size=input_size)  # P(Y|X,Z)
# modelY= train(input, targets, modelY, name="Y", optimize= "maximize")
modelY= train(input, targets, modelY, name="Y", optimize= "maximize", lambdaa= [0.1, 0])

/tmp/ipykernel_613967/3770493806.py:66: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  parents= torch.tensor(parents)


Epoch [50/1000], Loss: 0.5252 obsTVD:0.1473 iTVD:0.0007
Epoch [100/1000], Loss: 0.5097 obsTVD:0.1640 iTVD:0.0120
Epoch [150/1000], Loss: 0.5076 obsTVD:0.1627 iTVD:0.0112
Epoch [200/1000], Loss: 0.4975 obsTVD:0.1531 iTVD:0.0057
Epoch [250/1000], Loss: 0.4812 obsTVD:0.1365 iTVD:0.0108
Epoch [300/1000], Loss: 0.4627 obsTVD:0.1198 iTVD:0.0261
Epoch [350/1000], Loss: 0.4454 obsTVD:0.1038 iTVD:0.0382
Epoch [400/1000], Loss: 0.4396 obsTVD:0.0989 iTVD:0.0463
Epoch [450/1000], Loss: 0.4371 obsTVD:0.0966 iTVD:0.0485
Epoch [500/1000], Loss: 0.4373 obsTVD:0.0956 iTVD:0.0488
Epoch [550/1000], Loss: 0.4372 obsTVD:0.1009 iTVD:0.0499
Epoch [600/1000], Loss: 0.4353 obsTVD:0.0951 iTVD:0.0498
Epoch [650/1000], Loss: 0.4361 obsTVD:0.0955 iTVD:0.0516
Epoch [700/1000], Loss: 0.4355 obsTVD:0.0955 iTVD:0.0510
Epoch [750/1000], Loss: 0.4339 obsTVD:0.0939 iTVD:0.0486
Epoch [800/1000], Loss: 0.4337 obsTVD:0.0927 iTVD:0.0515
Epoch [850/1000], Loss: 0.4341 obsTVD:0.0948 iTVD:0.0534
Epoch [900/1000], Loss: 0.4335 o

In [ ]:
input= torch.cat([sampleZ, sampleX], dim=1)
targets = sampleY
input_size=4

modelY = SampleGenerator(input_size=input_size)  # P(Y|X,Z)
modelY= train(input, targets, modelY, name="Y", optimize= "minimize")

# Appendix

In [ ]:
# input= torch.cat([sampleZ, sampleX], dim=1)
input= None
targets = sampleZ
input_size=0

# Initialize the model, loss function, and optimizer
modelZ = SampleGenerator(input_size=input_size)

modelZ= train(input, targets, modelZ, name="Z")

In [ ]:
# input= torch.cat([sampleZ, sampleX], dim=1)
input= sampleZ
targets = sampleX
input_size=2

modelX = SampleGenerator(input_size=input_size)

modelX= train(input, targets, modelX, name='X')

# Generate

In [ ]:
outZ = modelZ(None, temperature=1.0, hard=True)
outt= outZ

outt= torch.argmax(outt, dim=1).view(-1,1)
unique_rows, counts = np.unique(np.array(outt.detach()), axis=0, return_counts=True)
joint_distribution = counts / len(outt)
joint_distribution

In [ ]:
X= torch.ones_like(sampleZ)
inputY= torch.cat([sampleZ, X], dim=1)
outY= modelY(inputY, temperature=1.0, hard=False)  


outt= outY
outt= torch.argmax(outt, dim=1).view(-1,1)
unique_rows, counts = np.unique(np.array(outt.detach()), axis=0, return_counts=True)
joint_distribution = counts / len(outt)
joint_distribution